In [4]:
%env LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/ubuntu/InterpolationRepair/spectra

env: LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/ubuntu/InterpolationRepair/spectra


## Test counterstrategy num states

In [16]:
import os
import re
import subprocess
import pandas as pd

PATH_TO_CLI = "../spectra/spectra-cli.jar"

def generate_counter_strat(specification):
    cmd = "java -jar {} -i {} --counter-strategy-jtlv-format".format(PATH_TO_CLI, specification)
    p = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True, timeout=600, text=True)
    output = p.stdout
    # Extract "Nodes num" and "Nodes table size"
    nodes_num = re.search(r'Nodes num: (\d+)', output).group(1)
    table_size = re.search(r'Nodes table size: (\d+)', output).group(1)

    # Use regular expression to find and extract the numbers after "State n:"
    state_numbers = re.findall(r'State (\d+)', output)

    # Convert the extracted numbers to integers
    state_numbers = [int(number) for number in state_numbers]

    # Find the largest value of n
    num_states = max(state_numbers)

    return int(nodes_num), int(table_size), num_states

def process_spectra_files(folder_path):
    data = []  # To store extracted data

    # Iterate through .spectra files in the folder
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(".spectra"):
            filepath = os.path.join(folder_path, filename)
            nodes_num, table_size, num_states = generate_counter_strat(filepath)
            data.append({
                "File Name": filename,
                "Nodes Num": nodes_num,
                "Table Size": table_size,
                "Number of States": num_states
            })

    # Create a DataFrame from the collected data
    df = pd.DataFrame(data)
    df = df.sort_values(by="File Name")
    return df


In [19]:
result_df = process_spectra_files("inputs/SYNTECH15-UNREAL-ORIGINAL")
result_df

In [17]:
result_df = process_spectra_files("outputs/BOOL-TRANSLATED-G")
result_df

,File Name,Nodes Num,Table Size,Number of States
0,ColorSortLTL3_689_ColorSort_fixed_unrealizable...,16290,119574,359
1,ColorSortLTLUnrealizable1_790_ColorSort_unreal...,3467,132860,70
2,ColorSortLTLUnrealizable2_791_ColorSort_unreal...,4746,93002,24
3,GyroUnrealizable_Var1_710_GyroAspect_unrealiza...,220,1022,3
4,GyroUnrealizable_Var2_710_GyroAspect_unrealiza...,224,1022,39
5,HumanoidLTL_458_Humanoid_fixed_unrealizable.sp...,324,1022,9
6,HumanoidLTL_503_Humanoid_fixed_unrealizable.sp...,949,3066,15
7,HumanoidLTL_531_Humanoid_unrealizable.spectra,869,10220,1476
8,HumanoidLTL_713_Humanoid_unrealizable.spectra,1596,11242,0
9,HumanoidLTL_741_Humanoid_unrealizable.spectra,1923,44968,3779
